#  Markov Decision Process

Written by [Junkun Yuan](https://junkunyuan.github.io/) (yuanjk0921@outlook.com).

See paper reading list and notes [here](https://junkunyuan.github.io/paper_reading_list/paper_reading_list.html).

Last updated on Aug 31, 2025; &nbsp; First committed on Aug 31, 2025.

**References**

- [**Hands-on RL**](https://github.com/boyu-ai/Hands-on-RL/blob/main/%E7%AC%AC2%E7%AB%A0-%E5%A4%9A%E8%87%82%E8%80%81%E8%99%8E%E6%9C%BA%E9%97%AE%E9%A2%98.ipynb)

**Contents**
- Markov Process
- Markov Reward Process
- Markov Decision Process
- Monte-Carlo Method
- Occupancy Measure
- Optimal Policy

## Markov Process

[**Stochastic process**](https://en.wikipedia.org/wiki/Stochastic_process) is a collection of random variables indexed by time or space that describes the evolution of a system subject to randomness. Let $S_t$ be the state set at timestep $t$, the state in the next timestep is represented as $P(S_{t+1}|S_1,...,S_t)$.

[**Markov property**](https://en.wikipedia.org/wiki/Markov_property) refers to the memoryless property of a stochastic process, which means that its future evolution is independent of its history. That is, $P(S_{t+1}|S_t)=P(S_{t+1}|S_1, ...,S_t)$.

[**Markov chain** or **Markov process**](https://en.wikipedia.org/wiki/Markov_chain) is a stochastic process with the Markov property. It can be represented as $<\mathcal{S}, \mathcal{P}>$, where $\mathcal{S}$ is a state set and $\mathcal{P}$ is a state transition matrix.

Given a Markov process, we can start at a state and continue to move forward and generate a **state episode**, this process is also called **sampling**.



## Markov Reward Process

A [**Markov reward model** or **Markov reward process (MRP)**](https://en.wikipedia.org/wiki/Markov_reward_model) is a stochastic process which extends Markov chain by adding a reward rate to each state. 

MRP consists of $<\mathcal{S}, \mathcal{P}, r, \gamma>$ where $r$ is a reward function and $\gamma\in[0,1)$ is a discount factor. A larger $\gamma$ pays more attention to long-term cumulative rewards, and vice versa.

A **Return** $G_t$ is the decaying cumulative reward: $G_t=R_t+\gamma R_{t+1}+\gamma^2 R_{t+2}+...=\sum_{k=0}^{\inf}\gamma^k R_{t+k}$, where $R_t$ is the obtained reward at timestep $t$. 

In [ ]:
## --------------------------------------------------------------------------------
## Compute return of Markov reward process
## --------------------------------------------------------------------------------
import numpy as np
np.random.seed(0)

rewards = [-1, -2, -2, 10, 1, 0]  # reward function
gamma = 0.5  # discount factor

def compute_return(start_index, chain, gamma):
    G = 0
    for i in reversed(range(start_index, len(chain))):
        G = gamma * G + rewards[chain[i] - 1]
    return G

chain = [1, 2, 3, 6]  # a state episode
start_index = 0
G = compute_return(start_index, chain, gamma)
print("The return of the state episode is %s." % G)
## --------------------------------------------------------------------------------

The return of the state episode is -2.5.


A **value function** evaluates **value**, i.e., the expected reward, of a state. That is, $V(s)=\mathbb{E}[G_t|S_t=s]$.

[**Bellman equation**](https://en.wikipedia.org/wiki/Bellman_equation) $V(s)=r(s)+\gamma\sum_{s'\in\mathcal{S}}p(s'|s)V(s')$ can be derived by:

\begin{equation}
\begin{aligned}
V(s)
&=\mathbb{E}[G_t|S_t=s] \\
&=\mathbb{E}[R_t+\gamma R_{t+1} + \gamma^2 R_{t+2}+...|S_t=s] \\
&=\mathbb{E}[R_t+\gamma G_{t+1}|S_t=s] \\
&=\mathbb{E}[R_t + \gamma V(S_{t+1})|S_t=s] \\
&=r(s)+\gamma\sum_{s'\in\mathcal{S}}p(s'|s)V(s')
\end{aligned}
\end{equation}

Let $\mathcal{S}=\{s_1,...,s_n\}$, $\mathcal{V}=[V(s_1),...,V(s_n)]^T$, $\mathcal{R}=[r(s_1),...,r(s_n)]^T$, the Bellman equation would be $\mathcal{V}=\mathcal{R}+\gamma\mathcal{P}\mathcal{V}$. 

We then have $O(n^3)$ compute complexity of the solution to value function: $\mathcal{V}=(I-\gamma\mathcal{P})^{-1}\mathcal{R}$.

In [ ]:
## --------------------------------------------------------------------------------
## Compute value by Bellman equation
## --------------------------------------------------------------------------------
def compute(P, rewards, gamma, states_num):
    rewards = np.array(rewards).reshape((-1, 1))
    value = np.dot(np.linalg.inv(np.eye(states_num, states_num) - gamma * P),
                   rewards)
    return value

## State transition matrix
P = [
    [0.9, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.6, 0.0, 0.4],
    [0.0, 0.0, 0.0, 0.0, 0.3, 0.7],
    [0.0, 0.2, 0.3, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
]
P = np.array(P)
V = compute(P, rewards, gamma, 6)
print("The value function is \n", V)
## --------------------------------------------------------------------------------

The value function is 
 [[-2.01950168]
 [-2.21451846]
 [ 1.16142785]
 [10.53809283]
 [ 3.58728554]
 [ 0.        ]]


## Markov Decision Process

[**Markov decision process (MDP)**](https://en.wikipedia.org/wiki/Markov_decision_process) (also called a stochastic dynamic program or stochastic control problem), is a model for sequential decision making when outcomes are uncertain.

MDP is consist of $<\mathcal{S},\mathcal{A},P,r, \gamma>$, where $\mathcal{S}$ is the state set, $\mathcal{A}$ is the action set, $\gamma$ is a discount factor, $r(s,a)$ is the reward function, $P(s'|s,a)$ is the state transition function.

The model in MDP is called **agent**, and its strategy to take action is called **policy**. That is, $\pi(a|s)=P(A_t=a|S_t=s)$.

The **state-value function** $V^{\pi}(s)$ is the expected return obtained by starting from state $s$. That is, $V^{\pi}(s)=\mathbb{E}_{\pi}[G_t|S_t=s]$.

The **action-value function** $Q^{\pi}(s,a)$ is the expected return obtained by starting from state $s$ and taking action $a$. That is, $Q^{\pi}(s,a)=\mathbb{E}_{\pi}[G_t|S_t=s,A_t=a]$.

We have their relationship: (1) $V^{\pi}(s) =\sum_{a\in\mathcal{A}} \pi(a|s) Q^{\pi}(s,a)$; (2) $Q^{\pi}(s,a)=r(s,a)+\gamma\sum_{s'\in\mathcal{S}}P(s'|s,a)V^{\pi}(s')$.

The **Bellman equaiton** is
$$
V^{\pi}(s)=\mathbb{E}_{\pi}[R_t+\gamma V^{\pi}(S_{t+1})|S_{t}=s]=\sum_{a\in\mathcal{A}}\pi(a|s)(r(s,a)+\gamma\sum_{s'\in S}p(s'|s,a)V^{\pi}(s')),
$$

$$
Q^{\pi}(s,a)=\mathbb{E}_{\pi}[R_t+\gamma Q^{\pi}(S_{t+1},A_{t+1})|S_t=s, A_t=a]=r(s,a)+\gamma\sum_{s'\in S}p(s'|s,a)\sum_{a'\in\mathcal{A}}\pi(a'|s')Q^{\pi}(s',a').
$$

In [14]:
## --------------------------------------------------------------------------------
## Calculate state-value function
## --------------------------------------------------------------------------------
S = ["s1", "s2", "s3", "s4", "s5"]  # state set
A = ["保持s1", "前往s1", "前往s2", "前往s3", "前往s4", "前往s5", "概率前往"]  # action set
## state transition function
P = {
    "s1-保持s1-s1": 1.0,
    "s1-前往s2-s2": 1.0,
    "s2-前往s1-s1": 1.0,
    "s2-前往s3-s3": 1.0,
    "s3-前往s4-s4": 1.0,
    "s3-前往s5-s5": 1.0,
    "s4-前往s5-s5": 1.0,
    "s4-概率前往-s2": 0.2,
    "s4-概率前往-s3": 0.4,
    "s4-概率前往-s4": 0.4,
}
## reward funciton
R = {
    "s1-保持s1": -1,
    "s1-前往s2": 0,
    "s2-前往s1": -1,
    "s2-前往s3": -2,
    "s3-前往s4": -2,
    "s3-前往s5": 0,
    "s4-前往s5": 10,
    "s4-概率前往": 1,
}
gamma = 0.5  # dicount factor
MDP = (S, A, P, R, gamma)

## Policy 1, random
Pi_1 = {
    "s1-保持s1": 0.5,
    "s1-前往s2": 0.5,
    "s2-前往s1": 0.5,
    "s2-前往s3": 0.5,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.5,
    "s4-概率前往": 0.5,
}
## Policy 2
Pi_2 = {
    "s1-保持s1": 0.6,
    "s1-前往s2": 0.4,
    "s2-前往s1": 0.3,
    "s2-前往s3": 0.7,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.1,
    "s4-概率前往": 0.9,
}

# 把输入的两个字符串通过“-”连接,便于使用上述定义的P、R变量
def join(str1, str2):
    return str1 + '-' + str2

## MRP from MDP + policy 1
P_from_mdp_to_mrp = [
    [0.5, 0.5, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.5, 0.5],
    [0.0, 0.1, 0.2, 0.2, 0.5],
    [0.0, 0.0, 0.0, 0.0, 1.0],
]
P_from_mdp_to_mrp = np.array(P_from_mdp_to_mrp)
R_from_mdp_to_mrp = [-0.5, -1.5, -1.0, 5.5, 0]

V = compute(P_from_mdp_to_mrp, R_from_mdp_to_mrp, gamma, 5)
print("The state-value function:\n", V)
## --------------------------------------------------------------------------------

The state-value function:
 [[-1.22555411]
 [-1.67666232]
 [ 0.51890482]
 [ 6.0756193 ]
 [ 0.        ]]


## Monte-Carlo Method

[**Monte-Carlo methods**](https://en.wikipedia.org/wiki/Monte_Carlo_method) are algorithms that rely on repeated random sampling to obtain numerical results.

The value can be directly estimated by the Monte-Carlo method: $V^{\pi}(s)=\mathbb{E}_{\pi}[G_t|S_t=s]\approx\frac{1}{N}\sum_{i=1}^{N}G_t^{(i)}$.

Another way is to maintain a state count and reward summation:
- Sample several sequence
- Update state count and reward summation of states in each sequence
- Estimate the value by averaging the reward of each state

In [12]:
def sample(MDP, Pi, timestep_max, number):
    S, A, P, R, gamma = MDP
    episodes = []
    for _ in range(number):
        episode = []
        timestep = 0
        ## randomly select a state as the start except for s5
        s = S[np.random.randint(4)]
        ## If the current state is the terminal state or the timestep is too long, finished
        while s != "s5" and timestep <= timestep_max:
            timestep += 1
            rand, temp = np.random.rand(), 0
            ## Select action based on policy
            for a_opt in A:
                temp += Pi.get(join(s, a_opt), 0)
                if temp > rand:
                    a = a_opt
                    r = R.get(join(s, a), 0)
                    break
            rand, temp = np.random.rand(), 0
            ## Transfer to the next state
            for s_opt in S:
                temp += P.get(join(join(s, a), s_opt), 0)
                if temp > rand:
                    s_next = s_opt
                    break
            episode.append((s, a, r, s_next))
            s = s_next
        episodes.append(episode)
    return episodes

## Sample 5 times
episodes = sample(MDP, Pi_1, 20, 5)
print('The 1-st episode\n', episodes[0])
print('The 4-th episode\n', episodes[3])

The 1-st episode
 [('s1', '前往s2', 0, 's2'), ('s2', '前往s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '前往s2', 0, 's2'), ('s2', '前往s3', -2, 's3'), ('s3', '前往s5', 0, 's5')]
The 4-th episode
 [('s4', '前往s5', 10, 's5')]


In [ ]:
## --------------------------------------------------------------------------------
## Estmate state value function by Monter-Carlo method
## --------------------------------------------------------------------------------
def MC(episodes, V, N, gamma):
    for episode in episodes:
        G = 0
        for i in range(len(episode) - 1, -1, -1):
            (s, a, r, s_next) = episode[i]
            G = r + gamma * G
            N[s] = N[s] + 1
            V[s] = V[s] + (G - V[s]) / N[s]

timestep_max = 20
## Sample 1000 times
episodes = sample(MDP, Pi_1, timestep_max, 1000)
V = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
N = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
MC(episodes, V, N, gamma=0.5)
print("The estimated state-value fucntion\n", V)
## --------------------------------------------------------------------------------

使用蒙特卡洛方法计算MDP的状态价值为
 {'s1': -1.221722518998472, 's2': -1.6913418637046331, 's3': 0.5057505430728482, 's4': 5.984319469137363, 's5': 0}


## Occupancy Measure

We define the probability of the agent to achieve state $s$ at timestep $t$ with policy $\pi$ is $P_t^{\pi}(s)$, then we have the MDP initial state distribution $P_0^{\pi}(s)=\nu^{\pi}_0(s)$, we have the **state visitation distribution**:
$$
\nu^{\pi}(s)=(1-\gamma)\sum_{t=0}^{\inf}\gamma^{t}P_{t}^{\pi}(s).
$$
It has the following property:
$$
\nu^{\pi}(s')=(1-\gamma)\nu_0(s')+\gamma\int P(s'|s,a)\pi(a|s)\nu^{\pi}(s)dsda.
$$
The **occupancy measure** is the the probability of action-state pair $(s,a)$, reprensented as 

$$
\rho^{\pi}(s,a)=(1-\gamma)\sum_{t=0}^{\inf}\gamma^{t}P_t^\pi(s)\pi(a|s).
$$ 

We have $\rho^{\pi}(s,a)=\nu^{\pi}(s)\pi(a|s)$.

**Theorem 1.** The occupancy measure $\rho^{\pi_1}$ and $\rho^{\pi_2}$ obtained by the agent interacting with the same MDP under different policies $\pi_1$ and $\pi_2$ satisfy: $\rho^{\pi_1}=\rho^{\pi_2}\Leftrightarrow\pi_1=\pi_2$.

**Theorem 2.** Given a legal occupancy measure $\rho$, the unique policy that generates it is $\pi_{\rho}=\frac{\rho(s,a)}{\sum_{a'}\rho(s,a')}$.

In [ ]:
## --------------------------------------------------------------------------------
## Estimate occupany
## --------------------------------------------------------------------------------
def occupancy(episodes, s, a, timestep_max, gamma):
    rho = 0
    total_times = np.zeros(timestep_max)  # 记录每个时间步t各被经历过几次
    occur_times = np.zeros(timestep_max)  # 记录(s_t,a_t)=(s,a)的次数
    for episode in episodes:
        for i in range(len(episode)):
            (s_opt, a_opt, r, s_next) = episode[i]
            total_times[i] += 1
            if s == s_opt and a == a_opt:
                occur_times[i] += 1
    for i in reversed(range(timestep_max)):
        if total_times[i]:
            rho += gamma**i * occur_times[i] / total_times[i]
    return (1 - gamma) * rho


gamma = 0.5
timestep_max = 1000

episodes_1 = sample(MDP, Pi_1, timestep_max, 1000)
episodes_2 = sample(MDP, Pi_2, timestep_max, 1000)
rho_1 = occupancy(episodes_1, "s4", "概率前往", timestep_max, gamma)
rho_2 = occupancy(episodes_2, "s4", "概率前往", timestep_max, gamma)
print(rho_1, rho_2)
## --------------------------------------------------------------------------------

0.112567796310472 0.23199480615618912


## Optimal Policy

**Optimial policy** $\pi$ satisfy $V^{\pi}(s)\geq V^{\pi'}(s)$ for any $\pi'$.

**Optimal state value function** is $V^*(s)=\max_{\pi}V^{\pi}(s)$; and **optimal value function** is $Q^*(s,a)=\max_{\pi}Q^{\pi}(s,a)$.

They have the relationship: $Q^*(s,a)=r(s,a)+\gamma\sum_{s'\in\mathcal{S}}P(s'|s,a)V^*(s')$. Meanwhile, $V^*(s)=\max_{a\in\mathcal{A}}Q^*(s,a)$.

We then have **Bellman optimally equation**:

$$
V^*(s)=\max_{a\in\mathcal{A}}\{r(s,a) + \gamma\sum_{s'\in\mathcal{S}}p(s'|s,a)V^*(s')\},
$$

$$
Q^*(s,a)=r(s,a)+\gamma\sum_{s'\in\mathcal{S}}p(s'|s,a)\max_{a'\in\mathcal{A}}Q^*(s',a').
$$